# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [105]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core import Model

import joblib

import requests

import numpy as np
import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'amazon_review_prediction_automl'

experiment=Experiment(ws, experiment_name)

found = False
dataset_name = "Amazon Reviews"
dataset_descriptions = "Set of Amazon products text reviews together with their rating value ranging from 1 to 5."

if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name] 

if not found:
        dataset = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 
        dataset = ds.register(workspace=ws, name=dataset_name, description=dataset_descriptions)

In [11]:
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "task": "classification",
    "primary_metric": "accuracy",
    "training_data": dataset,
    "label_column_name": "rating",
    "n_cross_validations":5,
    "compute_target": compute_target,
    "max_concurrent_iterations": 4,
    "iteration_timeout_minutes": 5,
    "iterations": 25    
}

automl_config = AutoMLConfig(**automl_settings)

In [18]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Exp-P3,AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-15T14:19:22.091836Z',
 'endTimeUtc': '2021-08-15T14:59:41.172059Z',
 'properties': {'num_iterations': '25',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-Exp-P3","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-154595","workspace_name":"quick-starts-ws-154595","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":25,"primary_metric":"accu

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [54]:
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)

print('\n Accuracy:', best_run_metrics['accuracy'])

Best Run Id:  AutoML_41b227b8-6de8-4d19-9b0f-0463a621bc87_23

 Accuracy: 0.8632642235928356


In [112]:
model_obj = remote_run.get_output()[1]

Package:azureml-automl-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-core, training version:1.33.0, current version:1.32.0
Package:azureml-dataprep, training version:2.20.1, current version:2.18.0
Package:azureml-dataprep-native, training version:38.0.0, current version:36.0.0
Package:azureml-dataprep-rslex, training version:1.18.0, current version:1.16.1
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-defaults, training version:1.33.0, current version:1.32.0
Package:azureml-interpret, training version:1.33.0, current version:1.32.0
Package:azureml-mlflow, training version:1.33.0, current version:1.32.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.32.0
Package:azureml-responsibleai, training version:1.33.0, current version:1.32.0
Package:azureml-telemetry, training version:1.33.0, current version:1.32.0
Package:azureml-train-automl-client, training version:1.33.0, current version:1.

In [113]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model_obj)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook154595/code/AMLE_P3'}

prefittedsoftvotingclassifier
{'estimators': ['9', '15', '18', '14', '8', '13', '6', '22', '17', '4'],
 'weights': [0.2,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.13333333333333333]}

9 - maxabsscaler
{'copy': True}

9 - logisticregression
{'C': 1.7575106248547894,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 '

In [100]:
#asdas = remote_run.get_output()

In [101]:
test_df = pd.DataFrame()
test_df["review"] = ["awful experience"]
asdas[1].predict(test_df)

array([5])

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [67]:
best_run.download_file("outputs/model.pkl")

In [68]:
model = Model.register(workspace=ws,
                       model_name='best_model_automl',                # Name of the registered model in your workspace.
                       model_path='./model.pkl',  # Local file to upload and register as a model.
                      )

Registering model best_model_automl


In [115]:
automl_env = Environment.get(ws, "AzureML-AutoML")
inference_config = InferenceConfig(source_directory=".", entry_script="score_automl.py", environment=automl_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2)

service_name = 'automl-model'

service = model.deploy(ws, service_name, [model], overwrite=True, inference_config=inference_config, deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-15 17:10:39+00:00 Creating Container Registry if not exists.
2021-08-15 17:10:39+00:00 Registering the environment.
2021-08-15 17:10:40+00:00 Use the existing image.
2021-08-15 17:10:41+00:00 Generating deployment configuration.
2021-08-15 17:10:41+00:00 Submitting deployment to compute..
2021-08-15 17:10:45+00:00 Checking the status of deployment automl-model..
2021-08-15 17:23:32+00:00 Checking the status of inference endpoint automl-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In the cell below, send a request to the web service you deployed to test it.

In [119]:
print(service.scoring_uri)

http://719fc540-7e77-46db-b677-7a512a1ccfbe.southcentralus.azurecontainer.io/score


In [124]:
reviews = [
    "i loved it", 
    "fine product",
    "I hate it",
    "awful"
]

predictions = requests.post("http://719fc540-7e77-46db-b677-7a512a1ccfbe.southcentralus.azurecontainer.io/score", json={
    "data": reviews
}).json()

pd.DataFrame(data={"Review":reviews, "Predicted Rating":predictions})

,Review,Predicted Rating
0,i loved it,5
1,fine product,5
2,I hate it,3
3,awful,1


In the cell below, print the logs of the web service and delete the service

In [118]:
print(service.get_logs())

2021-08-15T17:22:50,324653900+00:00 - iot-server/run 
2021-08-15T17:22:50,325340700+00:00 - gunicorn/run 
2021-08-15T17:22:50,326867900+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-15T17:22:50,351371000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3489174eb648a475780c9959ff366072/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-15T17:22:50,646920100+00:00 - iot-server/finish 1 0
2021-08-15T17:22:50,648343800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (73)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 102
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-15 17:22:52,987 | root | INFO | Starting up app insights client
logging socket was

In [ ]:
service.delete()